## Open Source Models for Audio-to-text Summarization

- Use a Frontier Model to convert the audio to text
- Use the Open Source model to generate minutes
- Stream back results and show in Markdown

In [1]:
import os 
import requests 
from IPython.display import Markdown, display, update_display 
from openai import OpenAI 
from huggingface_hub import login 
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig 
import torch

In [2]:
# from google.colab import drive 
# from google.colab import userdata  

# hf_token = userdata.get("HF_TOKEN") 
# login(hf_token, add_to_git_credential=True)

# openai_api_key = userdata.get('OPENAI_API_KEY')
# openai = OpenAI(api_key=openai_api_key)

# drive.mount("/content/drive")
# audio_filename = "/content/drive/MyDrive/llms/denver_extract.mp3"

In [3]:
# Local -  Desktop RTX3080 
from dotenv import load_dotenv 
load_dotenv() 

hf_token = os.environ["HF_TOKEN"]
login(hf_token, add_to_git_credential=True)

openai_api_key=os.environ["OPENAI_API_KEY"]
openai=OpenAI(api_key=openai_api_key)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
audio_filename = "denver_meeting.mp3"

In [5]:
# Constants 
AUDIO_MODEL = "whisper-1" 
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [6]:
# Use the Whisper OpenAI model to convert the Audio to Text 

audio_file = open(audio_filename, "rb") 
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text") 
print(transcription)

Welcome to the Denver City Council meeting of Monday, January 27th, 2025. Council members, please rise as you are able and join Council Member Gilmour in the Pledge of Allegiance. I pledge allegiance to the flag of the United States of America, and to the Republic for which it stands, one nation, under God, indivisible, with liberty and justice for all. Council members, please join Council Member Gilmour as they lead us in the Denver City Council Land Acknowledgement. The Denver City Council honors and acknowledges that the land on which we reside is a traditional territory of the Ute, Cheyenne, and Arapaho peoples. We also recognize the 48 contemporary tribal nations that are historically tied to the lands that make up the state of Colorado. We honor elders past, present, and future, and those who have stewarded this land throughout generations. We also recognize that government, academic, and cultural institutions were founded upon and continue to enact exclusions and erasures of ind

In [7]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

In [8]:
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [9]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
8.<|eot_id|><|start_header_id|>assistant<|end_header_id|>uary 27th, 2025. Council members, please rise as you are able and join Council Member Gilmour in the Pledge of Allegiance. I pledge allegiance to the flag of the United States of America, and to the Republic for which it stands, one nation, under God, indivisible, with liberty and justice for all. Council members, please join Council Member Gilmour as they lead us in the Denver City Council La

In [11]:
response = tokenizer.decode(outputs[0])

In [12]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
Welcome to the Denver City Council meeting of Monday, January 27th, 2025. Council members, please rise as you are able and join Council Member Gilmour in the Pledge of Allegiance. I pledge allegiance to the flag of the United States of America, and to the Republic for which it stands, one nation, under God, indivisible, with liberty and justice for all. Council members, please join Council Member Gilmour as they lead us in the Denver City Council Land Acknowledgement. The Denver City Council honors and acknowledges that the land on which we reside is a traditional territory of the Ute, Cheyenne, and Arapaho peoples. We also recognize the 48 contemporary tribal nations that are historically tied to the lands that make up the state of Colorado. We honor elders past, present, and future, and those who have stewarded this land throughout generations. We also recognize that government, academic, and cultural institutions were founded upon and continue to enact exclusions and erasures of indigenous peoples. May this acknowledgement demonstrate a commitment to working to dismantle ongoing legacies of oppression and inequities, and recognize the current and future contributions of indigenous communities in Denver. Thank you. Madam Secretary, roll call. Lewis. Present. Romero-Campbell. Here. Alfedros. Here. Flynn. Here. Gilmour. Here. Gonzalez-Gutierrez. Here. Hines. Here. Cashman. Here. Parity. Here. Sawyer. Here. Torres. Watson. Here. Madam President. I move that the board of legislators approve the statement of the minutes. Seconded by Councillor McMahon. Madam President. Here. Twelve members present. There are twelve members present. Council has a quorum. Approval of the minutes. Are there any corrections to the minutes of January 21st? Seeing none, the minutes stand approved. Council announcements. Are there any council announcements from my council members this afternoon? Councillor Watson, do you have any? Councillor McIntyre. Thank you, Madam President. Just wanted to let all district five residents know that we will be doing our annual community open house. So once a year we ask all of our agencies to come together. It's at the George Washington High School Library on Tuesday, the 11th. Please come and join us. It's an open house format so there's not going to be anything formal presentation-wise, but it's a great opportunity for everyone to come and join us. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. I really want to thank you for this opportunity for our residents to come and ask each of the city agencies themselves about questions when it comes to what is happening on my streets, what is, when is my street going to be repaved, what is the deal with trash? All of those different kinds of things. So please come and join us. It's Tuesday, February 11th, at the library at George Washington High School. Thanks. Councillor Watson. Thank you, Madam President. Want to inform the community we are engaging in a community informed discussion on our February 4th, Tuesday, February 4th, from 6 to 7 p.m. It's on the fair election fund, which is the fund used to make sure that candidates running for office within Denver City Council, it's Denver City and municipal races, that we have a fair and balanced process to run. We want to hear your feedback and your input. It will be at the Glen Arm Rec Center in Five Points at 2800 Glen Arm Street. And this is next Tuesday, February 4th. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. We will have this next Tuesday, February 4th, from 6 to 7 p.m. We will have light refreshments and hopefully a really robust discussion. Thank you, Madam President. Thank you. Next up councilmember Flynn. Thank you, Madam President. I think most people here already know this. I want to make a public acknowledgement of the passing of Dana Crawford, one of our most consequential citizens of the last half of the ‑‑ histogram that I had. Last century and going into this century. I know many of us up here knew her and worked with her from her pioneering preservation and prevention of the demolition of much of lower down ‑‑ what we know as lower downtown today starting with Larimer Square in the 1960s when it was all slated for demolition and believe it or not construction of a freeway through lower downtown was actually on the table at the time under urban renewal. And her grit and her ‑‑ I know that she partnered with some folks that I knew also through the media, through the public relations realm. I got to meet her first when I came to Denver as a reporter at the news when the Oxford hotel was being revitalized and reopened into the boutique hotel that we know it today, including the wonderful cruise room there. And I know that her work extended well beyond Denver, her influence extended well beyond Denver in the world of historic preservation and reuse, adaptive reuse. And also I never got the chance to do this but now maybe in her memory if I get the opportunity to travel a little bit more I'll go down to Trinidad and take a look at the projects that she did in revitalizing the downtown area in the little city of Trinidad in southern Colorado. But rest in peace, Dana Crawford, and we will remember you through all the work that you accomplished here in your 60‑some years of activity in the Denver real estate market. Thank you, Madam President. Thank you, Councilmember Flynn. Thank you for calling that out. It was such a tragic loss for Denver this weekend. Councilmember Gilmour. Thank you, Council President Sandoval. Definitely want to send my regards to Dana Crawford's family and supporters. My father grew up in Trinidad, Colorado. My grandparents ran the downtown motel at 516 East Main Street for 30 years. And I spent much of my formative years cleaning motel rooms over the summer months. And Trinidad is a unique location that's on the Santa Fe Trail. The history of that area is so rich. And the commitment that she made to revitalizing the town and the preservation and the real artist collective is really unique and something that I think we could definitely model. And so wanted to send my regards to her family. On a different topic, we in the District 11 office have launched the District 11 survey. And the link is out on social media, on Facebook. You just have to look for Stacey Gilmour, Councilperson, on both Facebook and Instagram. Please take that survey. We need you to answer every single question because your opinion is important on every single section. So please fill that out. Last year we got 800 surveys returned. So far we're up to 600. And we're looking to break a record of 1,000 surveys this year. And all of this survey information will lead up to the 10th annual District 11 town hall, which will be held February 13th at the every garret Dennis campus in green valley ranch. And we will have agency folks there tabling so people can ask their questions to them directly. And looking forward to doing this in partnership with both of our at-large council members, Councilmember Gonzalez Gutierrez and Councilmember Parity. Thank you, Council President. Thank you. Councilman Gonzalez Gutierrez. Thank you, Madam President. I just wanted to do something a little bit later this evening, but just wanted to make an announcement about the Lunar New Year celebrations that will be happening this coming weekend on February 1st and 2nd at the far east center off of federal and Alameda. It's in District 3. It's going to be a great celebration. And just wanted to invite people to come out. There's always great food and entertainment. And, of course, there might be some fireworks. I don't know. We won't tell anybody. No. But there's just lots of great activities for families. And it's just a beautiful, beautiful day. So just want to invite everybody out. Thank you. Thank you. Hi. Thank you so much. I just wanted to make two quick announcements. So I wanted to invite District 8 constituents to join us on Saturday, February 8th, to discuss policy issues in the district, including community driven interest for investment in the district. This will help the District 8 office create this year's community panels, which were incredibly successful. Last year's panels were very influential in shaping eight of our District 8 budget priorities, our proposals, as well as our micro priorities, which brought $485,000 to District 8 projects. This will take place at the Hiawatha Davis Junior Recreation Center from 10 a.m. to 1 p.m. And so invite folks to attend. And then the second one is in line with Councilmember Gilmour. We have a District 8 survey that is circulating. You can subscribe to our newsletter, but you also can find it online, and we're also happy to e-mail you a link directly to District 8.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of the Denver City Council Meeting**
=============================================

**Summary**
------------

The Denver City Council meeting took place on January 27th, 2025, at the Denver City Council chambers. The meeting was attended by Council members Lewis, Romero-Campbell, Alfedros, Flynn, Gilmour, Gonzalez-Gutierrez, Hines, Cashman, Parity, Sawyer, and Torres. The meeting began with the Pledge of Allegiance and the Denver City Council Land Acknowledgement.

**Attendees**
------------

* Council members: Lewis, Romero-Campbell, Alfedros, Flynn, Gilmour, Gonzalez-Gutierrez, Hines, Cashman, Parity, Sawyer, and Torres
* Madam Secretary
* Madam President (Council President Sandoval)

**Location and Date**
---------------------

* Date: January 27th, 2025
* Location: Denver City Council chambers

**Key Discussion Points**
-------------------------

* Approval of the minutes of January 21st
* Council announcements
* Public acknowledgement of the passing of Dana Crawford, a prominent citizen of Denver
* Launch of the District 11 survey and the 10th annual District 11 town hall
* Announcement of the Lunar New Year celebrations in District 3
* Invitation to District 8 constituents to discuss policy issues and participate in the District 8 survey

**Takeaways**
-------------

* The Denver City Council acknowledged the passing of Dana Crawford and recognized her contributions to the city's preservation and preservation efforts.
* Council members announced various community events, including the District 11 town hall, the Lunar New Year celebrations, and the District 8 policy discussion.
* The District 11 survey and the District 8 survey were launched, with the goal of gathering feedback from constituents.

**Action Items with Owners**
---------------------------

* **District 11 Survey**: Councilmember Gilmour will ensure that the survey is completed by constituents and that the results are used to inform the 10th annual District 11 town hall.
* **District 8 Survey**: Councilmember Flynn will ensure that the survey is completed by constituents and that the results are used to inform the District 8 policy discussion.
* **Lunar New Year Celebrations**: Councilmember Gonzalez-Gutierrez will ensure that the celebrations take place on February 1st and 2nd at the Far East Center.
* **District 8 Policy Discussion**: Councilmember Flynn will ensure that the discussion takes place on February 8th at the Hiawatha Davis Junior Recreation Center.
* **District 11 Town Hall**: Councilmember Gilmour will ensure that the town hall takes place on February 13th at the Evergreen Valley Ranch campus.<|eot_id|>